In [1]:
# SLM copy debug
import openpyxl 
import string
import pandas as pd
from pandas import ExcelWriter
import numpy as np
from openpyxl import load_workbook
from openpyxl import Workbook
from openpyxl import cell
from openpyxl.utils import get_column_letter
import xlsxwriter
import matplotlib.pyplot as plot
from os import listdir, walk
from os.path import isfile, join
import tkinter as tk
from tkinter import *
from win32com import client
import shutil


In [ ]:
### -=-=-=-=-=-=-=-=-= ###
# report generation
#### Constants - Does not change across reports ##### Report specific, NOT room specific
# Project Specific # 
Project_name = 'Hickam ASTM Testing'
Project_number = '23-001'
Client_name = 'Allied Pacific Builders'

# test date obtained from testplan list 
# same with source and receieve room names 
#test assembly
## Stock NIC notes -
NIC_exceed_vol = 'The receiver and/or source room had a volume exceeding 150 m3 (5,300 cu. ft.), and the absorption of the receiver and/or source room was greater than the maximum allowed per E336-16, Paragraph 9.4.1.2.'
NIC_not_enclosed_vol = 'The receiver and/or source room was not an enclosed space.'

NIC_less_than_minimum = 'The receiver and/or source room has a volume less than the minimum volume requirement of 25 m3 (883 cu. ft.).'
NIC_more_small_dims = 'The receiver and/or source room has one or more dimensions less than the minimum requirement of 2.3 m (7.5 ft.).'

## Stock ISR
ISR_reporting_only = 'The receiver room had a volume less than the minimum volume requirement of 40 m3.' 
ISR_less_thanmin = 'The receiver and/or source room has a volume less than the minimum volume requirement of 40 m3 (1413 cu. ft.).'

## Standards ##
astc_test_procedure = 'ASTM E336-16'
astc_test_blurb = 'Standard Test Method for Measurement of Airborne Sound Attenuation between Rooms in Buildings '

stc_calc_standard = 'ASTM E413-16'
stc_testblurb = 'Classification for Rating Sound Insulation '

AIIC_test_procedure = 'ASTM E1007-14'
AIIC_testblurb = 'Standard Test Method for Field Measurement of Tapping Machine Impact Sound Transmission Through Floor-Ceiling Assemblies and Associated Support Structure'

IIC_calc_standard = 'ASTM E989-06(2012)'
IIC_testblurb = 'Standard Classification for Determination of Impact Insulation Class (IIC)'

RT60_test_procedure = 'ASTM E2235-04(2012)'
RT60testblurb = 'Standard Test Method for Determination of Decay Rates for Use in Sound Insulation Test Methods'
## test procedure
test_procedure_pg = 'Determination of space-average sound pressure levels was performed via the manually scanned microphones techique, described in ' + astc_test_procedure + ', Paragraph 11.4.3.3.'+ 'The source room was selected in accordance with ASTM E336-11 Paragraph 9.2.5, which states that "If a corridor must be used as one of the spaces for measurement of ATL or FTL, it shall be used as the source space."'

# ASTC and NIC final result and blurb

# ASTC result, concat with this: 
ASTC_results_ATverbage = ' was calculated. The ASTC rating is based on Apparent Transmission Loss (ATL), and includes the effects of noise flanking. The ASTC reference contour is shown on the next page, and has been “fit” to the Apparent Transmission Loss values, in accordance with the procedure of'

#NIC results, concat with this:
NIC_results_NRverbage = ' was calculated. The NIC rating is based on Noise Reduction (NR), and includes the effects of noise flanking. The NIC reference contour is shown on the next page, and has been “fit” to the Apparent Transmission Loss values, in accordance with the procedure of'
# after results
results_blurb = 'The results stated in this report represent only the specific construction and acoustical conditions present at the time of the test. Measurements performed in accordance with this test method on nominally identical constructions and acoustical conditions may produce different results.'


In [ ]:
## # ## # # # # Now entering variables for the report path and raw datafiles
# short initial run
# testplan_path ='//DLA-04/Shared/KAILUA PROJECTS/2023/23-001 Hickam AFB Acoustical Testing/Documents/Python_devstuff/TestPlan_23-001_HickamPhase2_ASTM_testing_DnDexpedite.xlsx'
# test_list = pd.read_excel(testplan_path)
# testnums = test_list['Test Label']

# will enter all data when room vols and descriptions are entered into testplan 

## T:\KAILUA PROJECTS\2023\23-001 Hickam AFB Acoustical Testing\Test Data\Raw Data\SLM D - 3784
rawDtestpath = '//DLA-04/Shared/KAILUA PROJECTS/2023/23-001 Hickam AFB Acoustical Testing/Test Data/Raw Data/SLM D - 3784/'
rawEtestpath = '//DLA-04/Shared/KAILUA PROJECTS/2023/23-001 Hickam AFB Acoustical Testing/Test Data/Raw Data/SLM E - 4328/'
D_datafiles = [f for f in listdir(rawDtestpath) if isfile(join(rawDtestpath,f))]
E_datafiles = [f for f in listdir(rawEtestpath) if isfile(join(rawEtestpath,f))]
## T:\KAILUA PROJECTS\2023\23-001 Hickam AFB Acoustical Testing\Documents\Python_devstuff\Phase_2_reports\ASTC_reports
## DnDroom_ASTC_expedite T:\KAILUA PROJECTS\2023\23-001 Hickam AFB Acoustical Testing\Documents\Python_devstuff\Phase_2_reports\DnDroom_ASTC_expedite
# rawReportpath = '//DLA-04/Shared/KAILUA PROJECTS/2023/23-001 Hickam AFB Acoustical Testing/Documents/Python_devstuff/Phase_2_reports/ASTC_reports/DnDroom_ASTC_expedite/'

# rawReportpath = '//DLA-04/Shared/KAILUA PROJECTS/2023/23-001 Hickam AFB Acoustical Testing/Documents/Python_devstuff/Phase_2_reports/ASTC_reports/'
# reports =  [f for f in listdir(rawReportpath) if isfile(join(rawReportpath,f))]

# tables as images: 
#  =  pd.read_excel()
rawReportpath = '//DLA-04/Shared/KAILUA PROJECTS/2023/23-001 Hickam AFB Acoustical Testing/Documents/Python_devstuff/Phase_2_reports/ASTC_reports/'
reports =  [f for f in listdir(rawReportpath) if isfile(join(rawReportpath,f))]

In [2]:
def write_testdata(find_datafile, reportfile, newsheetname):
    if find_datafile[0] =='D':
        datafile_num = find_datafile[1:]
        datafile_num = '-831_Data.'+datafile_num+'.xlsx'
        slm_found = [x for x in D_datafiles if datafile_num in x]
        slm_found[0] = rawDtestpath+slm_found[0]# If this line errors, the test file is mislabled or doesn't exist 
        # print(srs_slm_found)
    elif find_datafile[0] == 'E':
        datafile_num = find_datafile[1:]
        datafile_num = '-831_Data.'+datafile_num+'.xlsx'
        slm_found = [x for x in E_datafiles if datafile_num in x]
        slm_found[0] = rawEtestpath+slm_found[0]# If this line errors, the test file is mislabled or doesn't exist 

    print(slm_found[0])

    srs_data = pd.read_excel(slm_found[0],sheet_name='OBA') # data must be in OBA tab
    with ExcelWriter(
    rawReportpath+reportfile,
    mode="a",
    engine="openpyxl",
    if_sheet_exists="replace",
    ) as writer:
        srs_data.to_excel(writer, sheet_name=newsheetname) #writes to report file

def write_RTtestdata(find_datafile, reportfile,newsheetname):
    if find_datafile[0] =='D':
        datafile_num = find_datafile[1:]
        datafile_num = '-RT_Data.'+datafile_num+'.xlsx'
        slm_found = [x for x in D_datafiles if datafile_num in x]
        slm_found[0] = rawDtestpath+slm_found[0]# If this line errors, the test file is mislabled or doesn't exist 
        # print(srs_slm_found)
    elif find_datafile[0] == 'E':
        datafile_num = find_datafile[1:]
        datafile_num = '-RT_Data.'+datafile_num+'.xlsx'
        slm_found = [x for x in E_datafiles if datafile_num in x]
        slm_found[0] = rawEtestpath+slm_found[0] # If this line errors, the test file is mislabled or doesn't exist 

    print(slm_found[0])

    srs_data = pd.read_excel(slm_found[0],sheet_name='Summary')# data must be in Summary tab for RT meas.
    # could reduce this function by also passing the sheet to be read into the args. 
    with ExcelWriter(
    rawReportpath+reportfile,
    mode="a",
    engine="openpyxl",
    if_sheet_exists="replace",
    ) as writer:
        srs_data.to_excel(writer, sheet_name=newsheetname) 

In [ ]:
# #IIC testlist only 
# # must have underscores in the report filenames around test label: _1.1.0_
# testplan_path ='TestPlan_23-010_SkyAlaMoana_IIC_testingv1.xlsx'
# rawReportpath = '//DLA-04/Shared/KAILUA PROJECTS/2023/23-010 Sky Ala Moana ASTM Testing/Documents/python_files/IIC_reports/tester/'
# reports =  [f for f in listdir(rawReportpath) if isfile(join(rawReportpath,f))]
# test_list = pd.read_excel(testplan_path)
# testnums = test_list['Test Label']
# # testnums = ['1.4.0']
# for i in range(len(testnums)):
#     curr_test = test_list.iloc[i]
#     find_report = curr_test['Test Label']
#     report_string = '_'+find_report+'_'
#     curr_report_file = [x for x in reports if report_string in x]
#     print(curr_report_file[0])

#     ## ASTC variables
#     find_source = curr_test['Source']
#     find_rec = curr_test['Recieve ']
#     find_BNL = curr_test['BNL']
#     find_RT = curr_test['RT']
    
#     ## IIC variables
#     find_posOne = curr_test['Position1']
#     find_posTwo = curr_test['Position2']
#     find_posThree = curr_test['Position3']
#     find_posFour = curr_test['Position4']
#     find_poscarpet = curr_test['Carpet']
#     find_Tapsrs = curr_test['SourceTap']

#     write_testdata(find_source,curr_report_file[0],'ASTC Source')
#     write_testdata(find_rec,curr_report_file[0],'ASTC Receive')
#     write_testdata(find_BNL,curr_report_file[0],'BNL')
#     write_RTtestdata(find_RT,curr_report_file[0],'RT')
#     write_testdata(find_posOne,curr_report_file[0],'AIIC POS 1')
#     write_testdata(find_posTwo,curr_report_file[0],'AIIC POS 2')
#     write_testdata(find_posThree,curr_report_file[0],'AIIC POS 3')
#     write_testdata(find_posFour,curr_report_file[0],'AIIC POS 4')
#     write_testdata(find_poscarpet,curr_report_file[0],'AIIC CARPET')
#     write_testdata(find_Tapsrs,curr_report_file[0],'AIIC Source')



In [ ]:
#ASTC testlist only 
# Hickam data paths - phase 2 
### T:\KAILUA PROJECTS\2023\23-001 Hickam AFB Acoustical Testing\Documents\Python_devstuff\Phase_2_reports
testplan_path ='//DLA-04/Shared/KAILUA PROJECTS/2023/23-001 Hickam AFB Acoustical Testing/Documents/Python_devstuff/TestPlan_23-001_HickamPhase2_ASTM_testing_DnDexpedite.xlsx'
test_list = pd.read_excel(testplan_path)
testnums = test_list['Test Label']
## T:\KAILUA PROJECTS\2023\23-001 Hickam AFB Acoustical Testing\Test Data\Raw Data\SLM D - 3784
rawDtestpath = '//DLA-04/Shared/KAILUA PROJECTS/2023/23-001 Hickam AFB Acoustical Testing/Test Data/Raw Data/SLM D - 3784/'
rawEtestpath = '//DLA-04/Shared/KAILUA PROJECTS/2023/23-001 Hickam AFB Acoustical Testing/Test Data/Raw Data/SLM E - 4328/'
D_datafiles = [f for f in listdir(rawDtestpath) if isfile(join(rawDtestpath,f))]
E_datafiles = [f for f in listdir(rawEtestpath) if isfile(join(rawEtestpath,f))]
## T:\KAILUA PROJECTS\2023\23-001 Hickam AFB Acoustical Testing\Documents\Python_devstuff\Phase_2_reports\ASTC_reports
## DnDroom_ASTC_expedite T:\KAILUA PROJECTS\2023\23-001 Hickam AFB Acoustical Testing\Documents\Python_devstuff\Phase_2_reports\DnDroom_ASTC_expedite
# rawReportpath = '//DLA-04/Shared/KAILUA PROJECTS/2023/23-001 Hickam AFB Acoustical Testing/Documents/Python_devstuff/Phase_2_reports/ASTC_reports/DnDroom_ASTC_expedite/'

rawReportpath = '//DLA-04/Shared/KAILUA PROJECTS/2023/23-001 Hickam AFB Acoustical Testing/Documents/Python_devstuff/Phase_2_reports/ASTC_reports/'
reports =  [f for f in listdir(rawReportpath) if isfile(join(rawReportpath,f))]
# must have underscores in the report filenames around test label: _1.1.0_

for i in range(len(testnums)):
    curr_test = test_list.iloc[i]
    find_report = curr_test['Test Label']
    report_string = '_'+find_report+'_'
    curr_report_file = [x for x in reports if report_string in x]
    print(curr_report_file[0])

    ## ASTC variables - test file numbers 
    find_source = curr_test['Source']
    find_rec = curr_test['Recieve ']
    find_BNL = curr_test['BNL']
    find_RT = curr_test['RT']
    # need to grab source/recieve vols and partition areas, then write it to a page to work out NIC/ASTC calcs
    # find_room_dims = curr_test['source_roomvol']
    
    write_testdata(find_source,curr_report_file[0],'ASTC Source')
    write_testdata(find_rec,curr_report_file[0],'ASTC Receive')
    write_testdata(find_BNL,curr_report_file[0],'BNL')
    write_RTtestdata(find_RT,curr_report_file[0],'RT')
    # future function - write room dimensions 
    # write_testdata(find_room_dims, curr_report_file[0],'RoomDims')


In [ ]:
# ## # make as many copies of the template file as there are entries in testnum ####
testplan_path ='//DLA-04/Shared/KAILUA PROJECTS/2023/23-001 Hickam AFB Acoustical Testing/Documents/Python_devstuff/TestPlan_23-001_HickamPhase2_ASTM_testingv3_complete.xlsx'
test_list = pd.read_excel(testplan_path)
testnums = test_list['Test Label'] ## Determines the labels and number of excel files copied
### Report Template path: 
rawReportpath = '//DLA-04/Shared/KAILUA PROJECTS/2023/23-001 Hickam AFB Acoustical Testing/Documents/Python_devstuff/Phase_2_reports/ASTC_reports/'
reports =  [f for f in listdir(rawReportpath) if isfile(join(rawReportpath,f))]
## Output folder location for excel report files: ###
outputfolder = '//DLA-04/Shared/KAILUA PROJECTS/2023/23-001 Hickam AFB Acoustical Testing/Documents/Python_devstuff/Phase_2_reports/All_reports/'

# print(testnums)
for i in range(len(testnums)):
    shutil.copy(rawReportpath+reports[0], outputfolder+'23-001 ASTC-NIC Test Report_'+testnums[i]+'_.xlsx')

In [4]:
#ASTC/NIC testlist only 
# Hickam data paths - phase 2 
### T:\KAILUA PROJECTS\2023\23-001 Hickam AFB Acoustical Testing\Documents\Python_devstuff\Phase_2_reports
# testplan_path ='//DLA-04/Shared/KAILUA PROJECTS/2023/23-001 Hickam AFB Acoustical Testing/Documents/Python_devstuff/TestPlan_23-001_HickamPhase2_ASTM_testing_DnDexpedite.xlsx'
testplan_path ='//DLA-04/Shared/KAILUA PROJECTS/2023/23-001 Hickam AFB Acoustical Testing/Documents/Python_devstuff/TestPlan_23-001_HickamPhase2_ASTM_testingv3_complete.xlsx'

test_list = pd.read_excel(testplan_path)
testnums = test_list['Test Label']
## T:\KAILUA PROJECTS\2023\23-001 Hickam AFB Acoustical Testing\Test Data\Raw Data\SLM D - 3784
rawDtestpath = '//DLA-04/Shared/KAILUA PROJECTS/2023/23-001 Hickam AFB Acoustical Testing/Test Data/Raw Data/SLM D - 3784/'
rawEtestpath = '//DLA-04/Shared/KAILUA PROJECTS/2023/23-001 Hickam AFB Acoustical Testing/Test Data/Raw Data/SLM E - 4328/'
D_datafiles = [f for f in listdir(rawDtestpath) if isfile(join(rawDtestpath,f))]
E_datafiles = [f for f in listdir(rawEtestpath) if isfile(join(rawEtestpath,f))]
## T:\KAILUA PROJECTS\2023\23-001 Hickam AFB Acoustical Testing\Documents\Python_devstuff\Phase_2_reports\ASTC_reports
## DnDroom_ASTC_expedite T:\KAILUA PROJECTS\2023\23-001 Hickam AFB Acoustical Testing\Documents\Python_devstuff\Phase_2_reports\DnDroom_ASTC_expedite
# rawReportpath = '//DLA-04/Shared/KAILUA PROJECTS/2023/23-001 Hickam AFB Acoustical Testing/Documents/Python_devstuff/Phase_2_reports/ASTC_reports/DnDroom_ASTC_expedite/'

rawReportpath = '//DLA-04/Shared/KAILUA PROJECTS/2023/23-001 Hickam AFB Acoustical Testing/Documents/Python_devstuff/Phase_2_reports/All_reports/'
reports =  [f for f in listdir(rawReportpath) if isfile(join(rawReportpath,f))]
# must have underscores in the report filenames around test label: _1.1.0_
project_name = '23-001_Hickam AFB Phase 7'
for i in range(len(testnums)):
    # list entry with all test data
    curr_test = test_list.iloc[i]
    
    find_report = curr_test['Test Label'] # using the test entry to find the report file name
    report_string = '_'+find_report+'_' 
    curr_report_file = [x for x in reports if report_string in x]
    print(curr_report_file[0]) #print the name of the report file being used

   ### IIC variables  #### When extending to IIC data
#     find_posOne = curr_test['Position1']
#     find_posTwo = curr_test['Position2']
#     find_posThree = curr_test['Position3']
#     find_posFour = curr_test['Position4']
#     find_poscarpet = curr_test['Carpet']
#     find_Tapsrs = curr_test['SourceTap']

#     write_testdata(find_source,curr_report_file[0],'ASTC Source')
#     write_testdata(find_rec,curr_report_file[0],'ASTC Receive')
#     write_testdata(find_BNL,curr_report_file[0],'BNL')
#     write_RTtestdata(find_RT,curr_report_file[0],'RT')
#     write_testdata(find_posOne,curr_report_file[0],'AIIC POS 1')
#     write_testdata(find_posTwo,curr_report_file[0],'AIIC POS 2')
#     write_testdata(find_posThree,curr_report_file[0],'AIIC POS 3')
#     write_testdata(find_posFour,curr_report_file[0],'AIIC POS 4')
#     write_testdata(find_poscarpet,curr_report_file[0],'AIIC CARPET')
#     write_testdata(find_Tapsrs,curr_report_file[0],'AIIC Source')


    ## ASTC variables - test file numbers 
    find_source = curr_test['Source']
    find_rec = curr_test['Recieve ']
    find_BNL = curr_test['BNL']
    find_RT = curr_test['RT']
    
    write_testdata(find_source,curr_report_file[0],'ASTC Source')
    write_testdata(find_rec,curr_report_file[0],'ASTC Receive')
    write_testdata(find_BNL,curr_report_file[0],'BNL')
    write_RTtestdata(find_RT,curr_report_file[0],'RT')


    raw_report = rawReportpath+curr_report_file[0]
   
    #### write room dimensions ####
    
    srs_roomName = curr_test['Source Room']
    rec_roomName = curr_test['Receiving Room']
    testdate = curr_test['Test Date'] 
    reportdate = curr_test['Report Date']
    source_vol = curr_test['source room vol']
    rec_vol = curr_test['receive room vol']
    partition_area = curr_test['partition area']
    partition_dim = curr_test['partition dim']
    source_rm_finish = curr_test['source room finish']
    rec_rm_finish = curr_test['receive room finish']
    srs_floor_descrip = curr_test['srs_floor']
    srs_ceiling_descrip = curr_test['srs_ceiling']
    srs_walls_descrip = curr_test['srs_Walls']
    rec_floor_descrip = curr_test['rec_floor']
    rec_ceiling_descrip = curr_test['rec_ceiling']
    rec_walls_descrip = curr_test['rec_Wall']
    tested_assem = curr_test['tested assembly']
    expected_perf = curr_test['expected performance']
    annex_two = curr_test['Annex 2 used?']
    test_assem_type = curr_test['Test assembly Type']
    # need some logic here to report only NIC or ASTC based on input, then assign verbage appropriately
    if source_vol or rec_vol >= 5300:
        NICreporting_Note = 'The receiver and/or source room had a volume exceeding 150 m3 (5,300 cu. ft.), and the absorption of the receiver and/or source room was greater than the maximum allowed per E336-16, Paragraph 9.4.1.2.'
    elif source_vol or rec_vol <= 833:
        NICreporting_Note = 'The receiver and/or source room has a volume less than the minimum volume requirement of 25 m3 (883 cu. ft.).'
    
    # append all this data or write individully to a dataframe, then save as new sheet
    # refer to this sheet with the SLM Data sheet to propigate to report.
    room_properties = pd.DataFrame(
        {
            "Source Room Name": srs_roomName,
            "Recieve Room Name": rec_roomName,
            "Testdate": testdate,
            "ReportDate": reportdate,
            "Test number": find_report,
            "Source Vol" : source_vol,
            "Recieve Vol": rec_vol,
            "Partition area": partition_area,
            "Partition dim.": partition_dim,
            "Source room Finish" : source_rm_finish,
            "Recieve room Finish": rec_rm_finish,
            "Srs Floor Descrip.": srs_floor_descrip,
            "Srs Ceiling Descrip.": srs_ceiling_descrip,
            "Srs Walls Descrip.": srs_walls_descrip,
            "Rec Floor Descrip.": rec_floor_descrip,
            "Rec Ceiling Descrip.": rec_ceiling_descrip,
            "Rec Walls Descrip.": rec_walls_descrip,          
            "Tested Assembly": tested_assem,
            "Expected Performance": expected_perf,
            "Annex 2 used?": annex_two,
            "Test assem. type": test_assem_type,
            "NIC reporting Note": NICreporting_Note
        },
        index=[0]
    )
    
    #### write this all to a proper reference template. ###
    with ExcelWriter(
    raw_report,
    mode="a",
    engine="openpyxl",
    if_sheet_exists="replace",
    ) as writer:
        room_properties.to_excel(writer, sheet_name='room_properties') 
    
    #### open excel file and write the reports to PDF ###
    # lol wil output wherever the file explorer in VS is currently. Probably should fix this. 
    excel = client.Dispatch("Excel.Application")
    # Read Excel File
    sheets = excel.Workbooks.Open(raw_report)
    
    ## need logic here to determine if NIC or ASTC, and report one or the other 
    if curr_test['NIC'] == 1 and curr_test['ASTC'] == 1:
        NIC_report = sheets.Worksheets[5]
    # needs to be closed and unique file name to work. 
    # Converting into PDF File
        NIC_report.ExportAsFixedFormat(0, project_name+report_string+'NIC_report.pdf')  # saves the NIC report workbook as PDF. 
        ASTC_report = sheets.Worksheets[6] # saves the ASTC report workbook.

        ASTC_report.ExportAsFixedFormat(0, project_name+report_string+'ASTC_report.pdf')  # saves the ASTC report workbook as PDF. 
        
    elif curr_test['ASTC'] == 1:
        ASTC_report = sheets.Worksheets[6] # saves the ASTC report workbook.

        ASTC_report.ExportAsFixedFormat(0, project_name+report_string+'ASTC_report.pdf')  # saves the ASTC report workbook as PDF. 
    elif curr_test['NIC'] == 1:
        NIC_report = sheets.Worksheets[5]
    # needs to be closed and unique file name to work. 
    # Converting into PDF File
        NIC_report.ExportAsFixedFormat(0, project_name+report_string+'NIC_report.pdf')  # saves the NIC report workbook as PDF. 



23-001 ASTC-NIC Test Report_2.1.1_.xlsx
//DLA-04/Shared/KAILUA PROJECTS/2023/23-001 Hickam AFB Acoustical Testing/Test Data/Raw Data/SLM D - 3784/831_0003784-20230426 183756-831_Data.667.xlsx
NICReport
ASTC Report
NIC Report
AIIC Report
ISR Report
//DLA-04/Shared/KAILUA PROJECTS/2023/23-001 Hickam AFB Acoustical Testing/Test Data/Raw Data/SLM E - 4328/831_0004328-20230426 183758-831_Data.164.xlsx
NICReport
ASTC Report
NIC Report
AIIC Report
ISR Report
//DLA-04/Shared/KAILUA PROJECTS/2023/23-001 Hickam AFB Acoustical Testing/Test Data/Raw Data/SLM E - 4328/831_0004328-20230426 183610-831_Data.163.xlsx
NICReport
ASTC Report
NIC Report
AIIC Report
ISR Report
//DLA-04/Shared/KAILUA PROJECTS/2023/23-001 Hickam AFB Acoustical Testing/Test Data/Raw Data/SLM E - 4328/831_0004328-20230425 082817-RT_Data.0000.xlsx
NICReport
ASTC Report
NIC Report
AIIC Report
ISR Report
NICReport
ASTC Report
NIC Report
AIIC Report
ISR Report
23-001 ASTC-NIC Test Report_2.1.10_.xlsx
//DLA-04/Shared/KAILUA PROJECTS

In [ ]:
## Single report tester 
testplan_path ='//DLA-04/Shared/KAILUA PROJECTS/2023/23-001 Hickam AFB Acoustical Testing/Documents/Python_devstuff/TestPlan_23-001_HickamPhase2_ASTM_testing_DnDexpedite_single.xlsx'
test_list = pd.read_excel(testplan_path)
testnums = test_list['Test Label']
# testnums = ['2.3.1']
for i in range(len(testnums)):
    # list entry with all test data
    curr_test = test_list.iloc[i]
    
    find_report = curr_test['Test Label'] # using the test entry to find the report file name
    report_string = '_'+find_report+'_' 
    curr_report_file = [x for x in reports if report_string in x]
    print(curr_report_file[0]) #print the name of the report file being used

    ## ASTC variables - test file numbers 
    find_source = curr_test['Source']
    find_rec = curr_test['Recieve ']
    find_BNL = curr_test['BNL']
    find_RT = curr_test['RT']
    # need to grab source/recieve vols and partition areas, then write it to a page to work out NIC/ASTC calcs
    # find_room_dims = curr_test['source_roomvol']
    
    write_testdata(find_source,curr_report_file[0],'ASTC Source')
    write_testdata(find_rec,curr_report_file[0],'ASTC Receive')
    write_testdata(find_BNL,curr_report_file[0],'BNL')
    write_RTtestdata(find_RT,curr_report_file[0],'RT')
    # future function - write room dimensions 
    # write_testdata(find_room_dims, curr_report_file[0],'RoomDims')
    raw_report = rawReportpath+curr_report_file[0]
    # test_instrumentation = pd.read_excel(raw_report,sheet_name='equipment')
    # write this all to a proper reference template.
    srs_roomName = curr_test['Source Room']
    rec_roomName = curr_test['Receiving Room']
    testdate = curr_test['Test Date'] 
    reportdate = curr_test['Report Date']
    source_vol = curr_test['source room vol']
    rec_vol = curr_test['receive room vol']
    partition_area = curr_test['partition area']
    partition_dim = curr_test['partition dim']
    source_rm_finish = curr_test['source room finish']
    rec_rm_finish = curr_test['receive room finish']
    srs_floor_descrip = curr_test['srs_floor']
    srs_ceiling_descrip = curr_test['srs_ceiling']
    srs_walls_descrip = curr_test['srs_walls']
    rec_floor_descrip = curr_test['rec_floor']
    rec_ceiling_descrip = curr_test['rec_ceiling']
    rec_walls_descrip = curr_test['rec_wall']
    tested_assem = curr_test['tested assembly']
    expected_perf = curr_test['expected performance']
    annex_two = curr_test['Annex 2 used?']
    test_assem_type = curr_test['Test assembly type']

    # append all this data or write individully to a dataframe, then save as new sheet
    # refer to this sheet with the SLM Data sheet to propigate to report.
    room_properties = pd.DataFrame(
        {
            "Source Room Name": srs_roomName,
            "Recieve Room Name": rec_roomName,
            "Testdate": testdate,
            "ReportDate": reportdate,
            "Test number": find_report,
            "Source Vol" : source_vol,
            "Recieve Vol": rec_vol,
            "Partition area": partition_area,
            "Partition dim.": partition_dim,
            "Source room Finish" : source_rm_finish,
            "Recieve room Finish": rec_rm_finish,
            "Srs Floor Descrip.": srs_floor_descrip,
            "Srs Ceiling Descrip.": srs_ceiling_descrip,
            "Srs Walls Descrip.": srs_walls_descrip,
            "Rec Floor Descrip.": rec_floor_descrip,
            "Rec Ceiling Descrip.": rec_ceiling_descrip,
            "Rec Walls Descrip.": rec_walls_descrip,          
            "Tested Assembly": tested_assem,
            "Expected Performance": expected_perf,
            "Annex 2 used?": annex_two,
            "Test assem. type": test_assem_type
        },
        index=[0]
    )
    
    

    with ExcelWriter(
    raw_report,
    mode="a",
    engine="openpyxl",
    if_sheet_exists="replace",
    ) as writer:
        room_properties.to_excel(writer, sheet_name='room_properties') 
    # read_excel cannot read in formulas. just paste in data to a new worksheet,
    # get the template to reference those values to calc the proper tables and grsaphs,
    # then output the pdfs each time with the code below


    # plot dataframe instrumentation
    # 
    # plot new dataframe of NIC results from Trnspose
    # if NIC  
    # plot dataframe of NIC results, read from Transpose
    # plot FR response of NIC: NOise reduction, NIC contour vs Frequency
    # freq = transpose_dataframe['']
    # if ASTC  
    # plot dataframe of ASTC results, read from Transpose
    # plot FR response of ASTC: Transmision loss, NIC contour vs Frequency


In [ ]:
#### Only exporting PDFs from reports ### 
#testplan path 
testplan_path ='//DLA-04/Shared/KAILUA PROJECTS/2023/23-001 Hickam AFB Acoustical Testing/Documents/Python_devstuff/TestPlan_23-001_HickamPhase2_ASTM_testingv2_complete.xlsx'
test_list = pd.read_excel(testplan_path)
testnums = test_list['Test Label']

# xls reports
rawReportpath = '//DLA-04/Shared/KAILUA PROJECTS/2023/23-001 Hickam AFB Acoustical Testing/Documents/Python_devstuff/Phase_2_reports/All_reports/'
reports =  [f for f in listdir(rawReportpath) if isfile(join(rawReportpath,f))]

project_name = '23-001_Hickam AFB Phase 7'

# excel = client.Dispatch("Excel.Application")
# # Read Excel File
# sheets = excel.Workbooks.Open(raw_report)
# NIC_report = sheets.Worksheets[5]
# # needs to be closed and unique file name to work. 
# # Converting into PDF File
# NIC_report.ExportAsFixedFormat(0, project_name+report_string+'NIC_report.pdf')  # saves the NIC report workbook as PDF. 

# ASTC_report = sheets.Worksheets[6] # saves the ASTC report workbook.

# ASTC_report.ExportAsFixedFormat(0, project_name+report_string+'ASTC_report.pdf')  # saves the ASTC report workbook as PDF. 
for i in range(len(testnums)):
    # list entry with all test data
    curr_test = test_list.iloc[i]
    #### open excel file and write the reports to PDF ###
    # lol wil output wherever the file explorer in VS is currently. Probably should fix this. 
    find_report = curr_test['Test Label'] # using the test entry to find the report file name
    report_string = '_'+find_report+'_' 
    curr_report_file = [x for x in reports if report_string in x]
    raw_report = rawReportpath+curr_report_file[0]
    excel = client.Dispatch("Excel.Application")
    # Read Excel File
    sheets = excel.Workbooks.Open(raw_report)
    # print(curr_test)
    ## need logic here to determine if NIC or ASTC, and report one or the other 
    if curr_test['NIC'] == 1 and curr_test['ASTC'] == 1:
        NIC_report = sheets.Worksheets[5]
    # needs to be closed and unique file name to work. 
    # Converting into PDF File
        NIC_report.ExportAsFixedFormat(0, project_name+report_string+'NIC_report.pdf')  # saves the NIC report workbook as PDF. 
        ASTC_report = sheets.Worksheets[6] # saves the ASTC report workbook.

        ASTC_report.ExportAsFixedFormat(0, project_name+report_string+'ASTC_report.pdf')  # saves the ASTC report workbook as PDF. 
        
    elif curr_test['ASTC'] == 1:
        ASTC_report = sheets.Worksheets[6] # saves the ASTC report workbook.

        ASTC_report.ExportAsFixedFormat(0, project_name+report_string+'ASTC_report.pdf')  # saves the ASTC report workbook as PDF. 
    elif curr_test['NIC'] == 1:
        NIC_report = sheets.Worksheets[5]
    # needs to be closed and unique file name to work. 
    # Converting into PDF File
        NIC_report.ExportAsFixedFormat(0, project_name+report_string+'NIC_report.pdf')  # saves the NIC report workbook as PDF. 


In [ ]:
if source_vol or rec_vol >= 5300:
    NICreporting_Note = 'The receiver and/or source room had a volume exceeding 150 m3 (5,300 cu. ft.), and the absorption of the receiver and/or source room was greater than the maximum allowed per E336-16, Paragraph 9.4.1.2.'
elif source_vol or rec_vol <= 833:
    NICreporting_Note = 'The receiver and/or source room has a volume less than the minimum volume requirement of 25 m3 (883 cu. ft.).'
    

In [ ]:
# ## # make as many copies of the template file as there are entries in testnum ####
testplan_path ='//DLA-04/Shared/KAILUA PROJECTS/2023/23-001 Hickam AFB Acoustical Testing/Documents/Python_devstuff/TestPlan_23-001_HickamPhase2_ASTM_testing_DnDexpedite.xlsx'
test_list = pd.read_excel(testplan_path)
testnums = test_list['Test Label']
rawReportpath = '//DLA-04/Shared/KAILUA PROJECTS/2023/23-001 Hickam AFB Acoustical Testing/Documents/Python_devstuff/Phase_2_reports/ASTC_reports/'
reports =  [f for f in listdir(rawReportpath) if isfile(join(rawReportpath,f))]
outputfolder = '//DLA-04/Shared/KAILUA PROJECTS/2023/23-001 Hickam AFB Acoustical Testing/Documents/Python_devstuff/Phase_2_reports/DnDroom_ASTC_expedite/'

# print(testnums)
for i in range(len(testnums)):
    shutil.copy(rawReportpath+reports[0], outputfolder+'23-001 ASTC-NIC Test Report_'+testnums[i]+'_.xlsx')

In [5]:
type(testplan_path)

str